In [4]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
from gtda.plotting import plot_heatmap
from sklearn.model_selection import train_test_split

In [5]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X shape: {X.shape}, y shape: {y.shape}")

/home/fenando/.local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


X shape: (70000, 784), y shape: (70000,)
X shape: (70000, 784), y shape: (70000,)


  #  X: Es un array o DataFrame que contiene las características (o atributos) de las imágenes. Cada fila de X representa una imagen, y cada columna representa un píxel de la imagen.

   # y: Es un array o Serie que contiene las etiquetas correspondientes a las imágenes en X. Cada etiqueta en y indica el dígito que se representa en la imagen correspondiente en X.

In [6]:
im8_idx = np.flatnonzero(y == "8")[0]
s = np.array(X.iloc[im8_idx]) 

img8 = s.reshape(28, 28)
plot_heatmap(img8)


In [7]:
X = np.array(X).reshape((-1, 28, 28))
train_size, test_size = 60, 10
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_size, test_size=test_size, stratify=y, random_state=666
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (60, 28, 28), y_train shape: (60,)
X_test shape: (10, 28, 28), y_test shape: (10,)


# Divide el conjunto de datos en conjuntos de entrenamiento y prueba

In [10]:
from gtda.images import Binarizer
im8_idx = np.flatnonzero(y_train == "8")[0]
im8 = X_train[im8_idx][None, :, :]

In [11]:
binarizer = Binarizer(threshold=0.4)
im8_binarized = binarizer.fit_transform(im8)

binarizer.plot(im8_binarized)

#Binariza la imagen del dígito "8" utilizando un umbral de 0.4 y muestra la imagen binarizada. Binarizar una imagen puede simplificar su representación al convertir los valores de píxeles a solo dos valores posibles, lo que puede ser útil en ciertos tipos de análisis o procesamiento de imágenes.


In [12]:
from gtda.images import RadialFiltration

radial_filtration = RadialFiltration(center=np.array([20, 6]))
im8_filtration = radial_filtration.fit_transform(im8_binarized)

radial_filtration.plot(im8_filtration, colorscale="jet")
     


# Se usa la filtración radial ya que es especifica o ideal para este problema
# realiza una filtración radial en una imagen binarizada 

In [13]:
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1)
im8_cubical = cubical_persistence.fit_transform(im8_filtration)

cubical_persistence.plot(im8_cubical)
     


# calculando la persistencia cubica

In [14]:
from gtda.diagrams import Scaler

scaler = Scaler()
im8_scaled = scaler.fit_transform(im8_cubical)

scaler.plot(im8_scaled)

In [15]:
from gtda.diagrams import HeatKernel

heat = HeatKernel(sigma=.15, n_bins=60, n_jobs=-1)
im8_heat = heat.fit_transform(im8_scaled)

# Visualise the heat kernel for H1
heat.plot(im8_heat, homology_dimension_idx=1, colorscale='jet')

In [16]:
from sklearn.pipeline import Pipeline
from gtda.diagrams import Amplitude

steps = [
    ("binarizer", Binarizer(threshold=0.4)),
    ("filtration", RadialFiltration(center=np.array([20, 6]))),
    ("diagram", CubicalPersistence()),
    ("rescaling", Scaler()),
    ("amplitude", Amplitude(metric="heat", metric_params={'sigma':0.15, 'n_bins':60}))
]

heat_pipeline = Pipeline(steps)

In [17]:

im8_pipeline = heat_pipeline.fit_transform(im8)
im8_pipeline
     

array([[0.57234335, 2.49781365]])